# Temperatura mínima

Encontrar a temperatura mínima medida em uma estação

## Conectando o Drive ao Colab

A primeira coisa que você quer fazer quando está trabalhando no Colab é montar seu Google Drive. Isso permitirá que você acesse qualquer diretório em seu Drive dentro do notebook Colab.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instalando o PySpark no Google Colab

Instalar o PySpark não é um processo direto como de praxe em Python. Não basta usar um pip install apenas. Na verdade, antes de tudo é necessário instalar dependências como o Java 8, Apache Spark 2.3.2 junto com o Hadoop 2.7.

In [2]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

A próxima etapa é configurar as variáveis de ambiente, pois isso habilita o ambiente do Colab a identificar corretamente onde as dependências estão rodando.

Para conseguir “manipular” o terminal e interagir como ele, você pode usar a biblioteca os.

In [3]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

## Implementando a aplicação

A primeira coisa a fazer é criar a aplicação e a sessão do Spark.

In [4]:
from pyspark.sql import SparkSession

def parse_line(line):
    fields = line.split(',')
    station_id = fields[0]
    entry_type = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return station_id, entry_type, temperature

spark = (SparkSession
         .builder
         .appName("MinTemperatures")
         .getOrCreate())

Carregar cada linha dos dados de origem em um RDD

In [5]:
lines = spark.sparkContext.textFile("/content/drive/MyDrive/Ensino/IFG/2022/01/Big_Data/pos-ia-bd/04-spark/data/1800.csv")

Use nossa função `parseLines` para converter em tuplas

In [6]:
parsedLines = lines.map(parse_line)

Filtre todas as entradas, usando TMIN

In [7]:
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])

Converte para (stationID, temperature)

In [8]:
stationTemps = minTemps.map(lambda x: (x[0], x[2]))

Reduza por stationID mantendo a temperatura mínima encontrada

In [9]:
minTemps = stationTemps.reduceByKey(lambda x, y: min(x, y))

Colete, formate e imprima os resultados

In [ ]:
results = minTemps.collect()

for result in results:
  print(result[0] + "\t{:.2f}F".format(result[1]))